---
### import

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import glob 
import cv2 
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
import tarfile

In [ ]:
# !pip install moviepy

In [ ]:
# !python.exe -m pip install --upgrade pip

In [ ]:
# !pip install ffmpeg --upgrade

In [ ]:
# !pip install --upgrade decorator==4.4.2

In [ ]:
# !pip uninstall moviepy decorator

^C


In [ ]:
def make_frame_dataset(json_path):  # return : (start_frame, end_frame, 말한 문장)
    with open(json_path, 'rb') as f: 
        data = json.load(f)

    fps = 25

    frame_datas = []
    sentence_infos = data[0].get('Sentence_info')
    for voice in sentence_infos:
        start_frame = int(voice['start_time']*fps)
        end_frame = int(voice['end_time']*fps)
        sentence_text = voice['sentence_text']
        frame_datas.append((start_frame, end_frame, sentence_text))
    return frame_datas

In [ ]:
def fps_29_to_25(video_path, fps=25): 
    video = VideoFileClip(video_path)
    video = video.set_fps(fps)
    # 변경된 속도로 비디오 파일 저장
    video.write_videofile(video_path.split('.')[0]+'_25fps.mp4', fps=fps)

In [ ]:
def cut_video_audio(video_path, start, end, video_save_path, audio_save_path):
    cap = cv2.VideoCapture(video_path)  # 영상용
    start -=15
    end += 15
    
    # 프레임 속성 알아내기
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fps = int(cap.get(cv2.CAP_PROP_FPS))

    cap.set(cv2.CAP_PROP_POS_FRAMES, start)  

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_save_path, fourcc, fps, (1080, 720))         # 여기서 512 로 맞춰줬음

    # 지정된 프레임 추출 및 저장
    for i in tqdm(range(end-start)):   
        ret, frame = cap.read()
        if ret:
            resized_frame = cv2.resize(frame, (int(frame_width//1.5), int(frame_height//1.5)))          # frame_width//2.109도 512에 맞춘거라 바꾸려면 바꿔야함 
            crop_frame = resized_frame[:,100:1180]
            out.write(crop_frame)
        else:
            break

    # 비디오 파일 닫기
    cap.release()
    out.release()

    # 비디오 파일 열기
    video = VideoFileClip(video_path)       # 비디오용

    # 프레임 범위에 해당하는 프레임 추출
    frames = video.subclip(start/video.fps, end/video.fps)
    # 추출한 프레임에 해당하는 오디오 데이터 추출
    sub_audio = frames.audio
    # 추출한 프레임과 오디오를 함께 저장
    sub_audio.write_audiofile(audio_save_path)


In [ ]:
def combine_vid_aud(vid_path, aud_path, out_path):
    # 비디오와 오디오 파일 열기
    video_clip = VideoFileClip(vid_path)
    audio_clip = AudioFileClip(aud_path)

    # 오디오와 비디오를 합치기
    final_clip = video_clip.set_audio(audio_clip)

    # 최종 파일로 내보내기
    final_clip.write_videofile(out_path, verbose=False, logger=None)

In [ ]:
def extract_Atypevideo_from_tar(tar_path, ex_path):
    with tarfile.open(tar_path, 'r') as tar: 
        video_names = tar.getmembers()
        for i in tqdm(video_names): 
            file_name = i.name
            if file_name.endswith('A_001.mp4') or file_name.endswith('C_002.mp4') or file_name.endswith('E_003.mp4') or file_name.endswith('G_004.mp4') or file_name.endswith('I_005.mp4') :
                tar.extract(tar.getmember(file_name), ex_path)
                os.rename(os.path.join(ex_path, file_name), os.path.join(ex_path, file_name.split('/')[-1]))

In [ ]:
def extract_Atypejson_from_tar(tar_path, ex_path):
    with tarfile.open(tar_path, 'r') as tar: 
        json_names = tar.getmembers()
        for i in tqdm(json_names): 
            file_name = i.name
            if file_name.endswith('A_001.json') or file_name.endswith('C_002.json') or file_name.endswith('E_003.json') or file_name.endswith('G_004.json') or file_name.endswith('I_005.json'):
                tar.extract(tar.getmember(file_name), ex_path)
                os.rename(os.path.join(ex_path, file_name), os.path.join(ex_path, file_name.split('/')[-1]))

---
### Tar 파일에서 A타입 mp4 파일만 뽑아내기

In [ ]:
%cd E:\

E:\


In [ ]:
ex_path = r'E:\dataset\source'   # 필요한 원본 파일들만 뽑아서 저장할 경로
if not os.path.exists(ex_path):
    os.makedirs(ex_path)

1, 53, 54, 55, 56, 57

In [ ]:
for TAR_NUM in [str(i) for i in range(2, 14)]: 
    VIDEO_FOLDER = 'TS'+TAR_NUM    # 원천 데이터 tar파일 이름
    VIDEO_PATH = 'E:\\009.립리딩(입모양) 음성인식 데이터\\01.데이터\\1.Training\\원천데이터\\'+VIDEO_FOLDER+'.tar'
    extract_Atypevideo_from_tar(VIDEO_PATH, ex_path)
    # 원본 라벨 tar파일에서 필요한(A방향) json만 뽑기
    JSON_FOLDER = 'TL'+TAR_NUM     # 라벨링 데이터 tar파일 이름
    JSON_PATH = 'E:\\009.립리딩(입모양) 음성인식 데이터\\01.데이터\\1.Training\\라벨링데이터\\'+JSON_FOLDER+'.tar'
    extract_Atypejson_from_tar(JSON_PATH, ex_path)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\009.립리딩(입모양) 음성인식 데이터\\01.데이터\\1.Training\\원천데이터\\TS2.tar'

In [ ]:
TAR_NUM = '53'

In [ ]:
# 원본 비디오 tar파일에서 필요한(A방향) 비디오만 뽑기
VIDEO_FOLDER = 'TS'+TAR_NUM    # 원천 데이터 tar파일 이름
VIDEO_PATH = 'E:\\009.립리딩(입모양) 음성인식 데이터\\01.데이터\\1.Training\\원천데이터\\'+VIDEO_FOLDER+'.tar'
extract_Atypevideo_from_tar(VIDEO_PATH, ex_path)

  0%|          | 0/293 [00:00<?, ?it/s]

In [ ]:
# 원본 라벨 tar파일에서 필요한(A방향) json만 뽑기
JSON_FOLDER = 'TL'+TAR_NUM     # 라벨링 데이터 tar파일 이름
JSON_PATH = 'E:\\009.립리딩(입모양) 음성인식 데이터\\01.데이터\\1.Training\\라벨링데이터\\'+JSON_FOLDER+'.tar'
extract_Atypejson_from_tar(JSON_PATH, ex_path)

  0%|          | 0/245 [00:00<?, ?it/s]

---
### 전처리(resize, 발화별 자르기) 후 비디오, 오디오 저장하기

In [ ]:
ex_path = 'E:\\alpaco\\data\\AI_hub'  
video_paths = sorted([os.path.join(ex_path, i) for i in  glob.glob1(ex_path, '*.mp4') if not i.endswith('fps.mp4')])    # 비디오 파일과 json 파일들 저장되어 있는 경로들
json_paths = sorted([os.path.join(ex_path, i) for i in  glob.glob1(ex_path, '*.json')])

In [ ]:
print(len(video_paths), len(json_paths))

195 195


In [ ]:
json_paths

['E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_A_012.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_C_005.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_G_006.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_A_010.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_C_003.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_G_001.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C235_A_011.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C235_C_004.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C235_G_002.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C245_A_009.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C245_C_005.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C245_G_006.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C253_A_008.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C253_C_004.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C253_G_003.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C254_A_003.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C254_C_007.json

In [ ]:
base_save_path = 'E:\\alpaco\\data'     # makeittalk, w2l 데이터 셋 저장 경로

w2l_path = os.path.join(base_save_path, 'W2L')
mit_path = os.path.join(base_save_path, 'MIT')

if not os.path.exists(w2l_path):
    os.mkdir(w2l_path)
if not os.path.exists(mit_path):
    os.mkdir(mit_path)

In [ ]:
json_paths[1:5]


['E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_C_005.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_G_006.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_A_010.json',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_C_003.json']

In [ ]:
video_paths[1:5]

['E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_C_005.mp4',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C190_G_006.mp4',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_A_010.mp4',
 'E:\\alpaco\\data\\AI_hub\\lip_J_1_F_04_C210_C_003.mp4']

In [ ]:
start_idx = 178

for folder_idx, (v_p, j_p) in enumerate(zip(video_paths[start_idx:], json_paths[start_idx:])):                  ### 중간에 오류나거나 끊겨서 다시 해야하면 여기 수정해서 이미 했던 것 넘기고 하고
    fps_29_to_25(v_p)     # 5분 짜리 30프레임 -> 25프레임(약 17분 걸림) 
    print(folder_idx+1)
    # 나머지 과정은 7분정도 걸림
    frame_data = make_frame_dataset(j_p)
    v_p = v_p.replace('.mp4', '_25fps.mp4')
    
    folder_idx += start_idx+1                                                                       ### 여기 폴더 인덱스 잘 수정해서 해야 이미 했던 폴더이름 안겹치게 해서 
    
    if not os.path.exists(os.path.join(w2l_path, f'{folder_idx:03d}')):                 
        os.makedirs(os.path.join(w2l_path, f'{folder_idx:03d}'))
    if not os.path.exists(os.path.join(mit_path, f'{folder_idx:03d}')):
        os.makedirs(os.path.join(mit_path, f'{folder_idx:03d}'))

    for file_idx, (s, e, text) in enumerate(frame_data):           
        file_idx+=1

        w2l_save_path = os.path.join(w2l_path, f'{folder_idx:03d}', f'{file_idx:05d}')       ### 여기 밑에들 폴더 이름 잘 수정해서 해야 이미 했던 폴더에 안 덮어씌워짐

        video_save_path = os.path.join(mit_path, f'{folder_idx:03d}', f'{file_idx:05d}.mp4')
        audio_save_path = os.path.join(mit_path, f'{folder_idx:03d}', f'{file_idx:05d}.wav')

        cut_video_audio(v_p, s, e, video_save_path, audio_save_path)
        
        combine_vid_aud(video_save_path, audio_save_path, w2l_save_path+'.mp4')
        with open(w2l_save_path+'.txt', 'wt') as f: 
            f.write(f'''Text : {text.rstrip('.')}''')

In [ ]:
print('hello world')

hello world


In [26]:
import os
import cv2
import numpy as np

In [32]:
def convert(video_dir, out_dir, max_num_frames=2000):
    ret, fl2d, fl3d = __video_facial_landmark_detection__(video_dir= video_dir, display=False, max_num_frames=max_num_frames)
    if (not ret):
        return
    if (len(fl3d) < 9):
        print('The length of the landmark is too short, skip')
        return

    # Step 3: raw save landmark / audio
    fl3d = np.array(fl3d)
    np.savetxt(out_dir,
                fl3d, fmt='%.2f')

base = r'E:\alpaco\data\MIT\078'
for video in os.listdir(base):
    if video[-3:] == 'mp4':
        video_dir = os.path.join(base, video)
        out_dir = base + '_' + video[:-3] + 'txt'
        # print(out_dir)
        convert(video_dir, out_dir)
        break

E:\alpaco\data\MIT\078_00001.txt
video_dir : E:\alpaco\data\MIT\078\00001.mp4
Process Video E:\alpaco\data\MIT\078\00001.mp4, len: 185, FPS: 25.00, W X H: 1080 x 720
	 ==> Final processed frames 185/185


In [20]:


def __video_facial_landmark_detection__(video_dir=None, display=False, WriteFileName='_fl_detect.mp4',
                                        max_num_frames=250, write=False):

    # load video
    print('video_dir : ' + video_dir)
    video = cv2.VideoCapture(video_dir)

    # return false if cannot open
    if (video.isOpened() == False):
        print('Unable to open video file')
        return False, None

    # display info
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print('Process Video {}, len: {}, FPS: {:.2f}, W X H: {} x {}'.format(video_dir, length, fps, w, h))

    video_facial_landmark = []  # face-landmark np array per frame =: idx + [x,y] * 68
    video_facial_landmark_3d = []  # face-landmark np array per frame =: idx + [x,y,z] * 68
    frame_id = 0
    not_detected_frames = 0

    # gpu_frame = cv2.cuda_GpuMat()   # gpu


    while (video.isOpened()):
        ret, frame = video.read()

        # gpu_frame.upload(frame)  # gpu


        # reach EOF
        if (ret == False):
            break

        # too many not-detected frames (in middle of video)
        if (not_detected_frames > 5):
            if (len(video_facial_landmark) < 10):
                # at beginning of the video
                video_facial_landmark = []
                video_facial_landmark_3d = []
            else:
                break

        # dlib facial landmark detect
        
        # img_ret, shape, shape_3d = __image_facial_landmark_detection__(img=gpu_frame)   # gpu
        img_ret, shape, shape_3d = __image_facial_landmark_detection__(img=frame)   # cpu

        # successfully detected
        if (img_ret):
            # print('\t ==> frame {}/{}'.format(frame_id, length))

            # current frame xy coordinates
            xys = []
            for part_i in range(68):
                xys.append(shape.part(part_i).x)
                xys.append(shape.part(part_i).y)

            # check any not_detected_frames, and interp them
            if (not_detected_frames > 0 and len(video_facial_landmark) > 0):
                # interpolate
                def interp(last, cur, num, dims=68 * 2 + 1):
                    interp_xys_np = np.zeros((num, dims))
                    for dim in range(dims):
                        interp_xys_np[:, dim] = np.interp(np.arange(0, num), [-1, num], [last[dim], cur[dim]])
                    interp_xys_np = np.round(interp_xys_np).astype('int')
                    interp_xys = [list(xy) for xy in interp_xys_np]
                    return interp_xys

                interp_xys = interp(video_facial_landmark[-1], [frame_id] + xys, not_detected_frames)
                video_facial_landmark += interp_xys

            not_detected_frames = 0

            # save landmark/frame_index
            video_facial_landmark.append([frame_id] + xys)
            if (shape_3d.any()):
                video_facial_landmark_3d.append([frame_id] + list(np.reshape(shape_3d, -1)))


        else:
            print('\t ==> frame {}/{} Not detected'.format(frame_id, length))
            not_detected_frames += 1
        
        frame_id += 1

        if(frame_id > max_num_frames):
            break

    video.release()
 
    cv2.destroyAllWindows()

    print('\t ==> Final processed frames {}/{}'.format(frame_id, length))

    return True, video_facial_landmark, video_facial_landmark_3d

In [8]:
# !pip install face_alignment

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/23.7 MB ? eta -:--:--
                                              0.5/23.7 MB 15.5 MB/s eta 0:00:02
     -                                        1.0/23.7 MB 12.9 MB/s eta 0:00:02
     --                                       1.6/23.7 MB 12.4 MB/s eta 0:00:02
     ---                                      2.1/23.7 MB 12.1 MB/s eta 0:00:02
     ----                                     2.6/23.7 MB 12.9 MB/s eta 0:00:02
     -----                                    3.1/23.7 MB 12.5 MB/s eta 0:00:02
     ------                                   3.7/23.7 MB 12.4 MB/s eta 0:00:02
     -------                                  4.2/23.7 MB 12.3 MB/s eta 0:00:02
     --------                                 4.8/23.7 MB 12.1 MB/s eta 0:00:02
     --------                                 5.3/23.7 MB 12.1 MB/s eta 0:00:02
     ---------            

In [22]:
import face_alignment

predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cuda', flip_input=True)

def __image_facial_landmark_detection__(img=None):
        shapes = predictor.get_landmarks(img)
        if (not shapes):
            return False, None, None

        max_size_idx = 0
        shape = ShapeParts(shapes[max_size_idx][:, 0:2])
        shape_3d = shapes[max_size_idx]

        # when use 2d estimator
        shape_3d = np.concatenate([shape_3d, np.ones(shape=(68, 1))], axis=1)

        return True, shape, shape_3d

In [24]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class ShapeParts:
    def __init__(self, np_pts):
        self.data = np_pts

    def part(self, idx):
        return Point(self.data[idx, 0], self.data[idx, 1])